In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oliverli0414","key":"19df8e5218b8495a8c1ea362134c4a27"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c home-credit-default-risk
!unzip home-credit-default-risk.zip

 97% 669M/688M [00:04<00:00, 206MB/s]
100% 688M/688M [00:04<00:00, 155MB/s]
Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [4]:
import pandas as pd
import numpy as np

In [14]:
application_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')

In [19]:
print(POS_CASH_balance.isna().sum())
print(POS_CASH_balance['NAME_CONTRACT_STATUS'].value_counts())

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64
Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: NAME_CONTRACT_STATUS, dtype: int64


In [9]:
POS_CASH_balance_sorted = pd.DataFrame()
POS_CASH_balance_sorted = POS_CASH_balance.groupby('SK_ID_PREV').max('MONTHS_BALANCE')[['SK_ID_CURR','MONTHS_BALANCE','CNT_INSTALMENT','CNT_INSTALMENT_FUTURE']]
POS_CASH_balance_sorted['OCCURENCES']=POS_CASH_balance.groupby('SK_ID_PREV').count()['MONTHS_BALANCE']
POS_CASH_balance_sorted['DISTINCT_CNT_INSTALMENT'] = POS_CASH_balance.groupby('SK_ID_PREV')['CNT_INSTALMENT'].nunique()
POS_CASH_balance_sorted['COUNT_DEMAND']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Demand'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_COMPLETED']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Completed'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_SIGNED']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Signed'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_ACTIVE']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Active'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_RETURNED']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Returned to the store'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_APPROVED']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Approved'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_AMORTIZED']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Amortized debt'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_CANCELED']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Canceled'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted['COUNT_XNA']=POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='XNA'].groupby('SK_ID_PREV')['NAME_CONTRACT_STATUS'].count()
POS_CASH_balance_sorted[['COUNT_DEMAND','COUNT_COMPLETED','COUNT_SIGNED','COUNT_ACTIVE','COUNT_RETURNED',
                         'COUNT_APPROVED','COUNT_AMORTIZED','COUNT_CANCELED','COUNT_XNA']]=POS_CASH_balance_sorted[['COUNT_DEMAND','COUNT_COMPLETED','COUNT_SIGNED','COUNT_ACTIVE','COUNT_RETURNED',
                         'COUNT_APPROVED','COUNT_AMORTIZED','COUNT_CANCELED','COUNT_XNA']].fillna(0)
POS_CASH_balance_sorted['PERC_DEMAND']=POS_CASH_balance_sorted['COUNT_DEMAND']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_COMPLETED']=POS_CASH_balance_sorted['COUNT_COMPLETED']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_SIGNED']=POS_CASH_balance_sorted['COUNT_SIGNED']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_ACTIVE']=POS_CASH_balance_sorted['COUNT_ACTIVE']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_RETURNED']=POS_CASH_balance_sorted['COUNT_RETURNED']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_APPROVED']=POS_CASH_balance_sorted['COUNT_APPROVED']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_AMORTIZED']=POS_CASH_balance_sorted['COUNT_AMORTIZED']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_CANCELED']=POS_CASH_balance_sorted['COUNT_CANCELED']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['PERC_XNA']=POS_CASH_balance_sorted['COUNT_XNA']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['SUM_SK_DPD'] = POS_CASH_balance.groupby('SK_ID_PREV')['SK_DPD'].sum()
POS_CASH_balance_sorted['AVG_SK_DPD'] = POS_CASH_balance_sorted['SUM_SK_DPD']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_sorted['SUM_SK_DPD_DEF'] = POS_CASH_balance.groupby('SK_ID_PREV')['SK_DPD_DEF'].sum()
POS_CASH_balance_sorted['AVG_SK_DPD_DEF'] = POS_CASH_balance_sorted['SUM_SK_DPD_DEF']/POS_CASH_balance_sorted['OCCURENCES']
POS_CASH_balance_ID_CURR = POS_CASH_balance_sorted.groupby('SK_ID_CURR').sum()
POS_CASH_balance_ID_CURR = POS_CASH_balance_ID_CURR.drop('MONTHS_BALANCE',axis=1)

In [10]:
POS_CASH_balance_ID_CURR__application_train = POS_CASH_balance_ID_CURR.merge(application_train, on='SK_ID_CURR', how='inner')
print(POS_CASH_balance_ID_CURR__application_train)

        SK_ID_CURR  CNT_INSTALMENT  CNT_INSTALMENT_FUTURE  OCCURENCES  \
0           100002            24.0                   24.0          19   
1           100003            30.0                   30.0          28   
2           100004             4.0                    4.0           4   
3           100006            72.0                   72.0          21   
4           100007            76.0                   76.0          66   
...            ...             ...                    ...         ...   
289439      456251             8.0                    8.0           9   
289440      456252             6.0                    6.0           7   
289441      456253            22.0                   12.0          17   
289442      456254            30.0                   30.0          20   
289443      456255           114.0                  114.0          71   

        DISTINCT_CNT_INSTALMENT  COUNT_DEMAND  COUNT_COMPLETED  COUNT_SIGNED  \
0                             1           0

In [12]:
corr = POS_CASH_balance_ID_CURR__application_train.corr(method='spearman')
print(corr)

                         SK_ID_CURR  CNT_INSTALMENT  CNT_INSTALMENT_FUTURE  \
SK_ID_CURR                 1.000000        0.003182               0.003260   
CNT_INSTALMENT             0.003182        1.000000               0.996090   
CNT_INSTALMENT_FUTURE      0.003260        0.996090               1.000000   
OCCURENCES                 0.003905        0.863999               0.859946   
DISTINCT_CNT_INSTALMENT    0.002697        0.827562               0.820372   
COUNT_DEMAND               0.003700       -0.000314              -0.002489   
COUNT_COMPLETED            0.003882        0.678478               0.666808   
COUNT_SIGNED              -0.001289        0.181006               0.181688   
COUNT_ACTIVE               0.003823        0.861883               0.858860   
COUNT_RETURNED            -0.003627        0.022891               0.022810   
COUNT_APPROVED             0.002314        0.066600               0.067136   
COUNT_AMORTIZED           -0.001308       -0.003016             

In [13]:
corr.to_excel(r'POS_CASH_balance.spearman.xlsx',index=True,header=True)

Choose SUM_SK_DPD_DEF and Occurences due to correlation.
Others with high correlation are already related.